# Predicting the Solubility of Small Molecules using DeepChem

## Overview

This tutorial demonstrates how to use DeepChem to build models for predicting small molecule solubility. We'll explore how to work with molecular data and implement deep learning models for chemical property prediction.

## Learning Objectives

- Learn how to use DeepChem for molecular property prediction
- Understand how to process and prepare molecular data for machine learning
- Build and train deep learning models for solubility prediction
- Evaluate model performance on chemical datasets

### Tasks to complete

- Load and process molecular data
- Build DeepChem model
- Train the model on solubility data
- Evaluate predictions and model performance

## Prerequisites

- A working Python environment and familiarity with Python
- Basic understanding of machine learning concepts
- Familiarity with pandas and numpy libraries
- Knowledge of basic statistical concepts

## Get Started

- Please select "conda_tensorflow2_p310" kernel from SageMake Jupyter-lab notebook.

### Import necessary libraries

Note that you will likely get some warnings about missing dependencies and removed features.  This is expected since we aren't using the full capabilities of deepchem in this tutorial.

In [ ]:
!pip install --pre deepchem[tensorflow]

In [ ]:
import deepchem as dc
import warnings
warnings.filterwarnings('ignore')
dc.__version__

## Training a Model with DeepChem

Deep learning can be used to solve many sorts of problems, but the basic workflow is usually the same.  Here are the typical steps you follow.

1. Select the data set you will train your model on (or create a new data set if there isn't an existing suitable one).
2. Create the model.
3. Train the model on the data.
4. Evaluate the model on an independent test set to see how well it works.
5. Use the model to make predictions about new data.

With DeepChem, each of these steps can be as little as one or two lines of Python code.  In this tutorial we will walk through a basic example showing the complete workflow to solve a real world scientific problem.

The problem we will solve is predicting the solubility of small molecules given their chemical formulas.  This is a very important property in drug development: if a proposed drug isn't soluble enough, you probably won't be able to get enough into the patient's bloodstream to have a therapeutic effect.  

The first thing we need is a data set of measured solubilities for real molecules.  One of the core components of DeepChem is MoleculeNet, a diverse collection of chemical and molecular data sets.  For this tutorial, we can use the Delaney solubility data set. The property of solubility in this data set is reported in log(solubility) where solubility is measured in moles/liter.

In [ ]:
# The Delaney (ESOL) dataset a regression dataset containing structures and
# water solubility data for 1128 compounds. The dataset is widely used to
# validate machine learning models on estimating solubility directly from
# molecular structures (as encoded in SMILES strings).
# featurizer: the featurizer to use for processing the data.
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer="GraphConv")
train_dataset, valid_dataset, test_dataset = datasets

In [ ]:
print(test_dataset.ids)

First, notice the `featurizer` argument passed to the `load_delaney()` function.  Molecules can be represented in many ways.  We therefore tell it which representation we want to use, or in more technical language, how to "featurize" the data.  Second, notice that we actually get three different data sets: a training set, a validation set, and a test set.  Each of these serves a different function in the standard deep learning workflow.

### Create model

Now that we have our data, the next step is to create a model.  We will use a particular kind of model called a "graph convolutional network", or "graphconv" for short.

In [ ]:
# Build Graph Convolutional Models.
# These graph convolutions start with a per-atom set of
# descriptors for each atom in a molecule, then combine and recombine these
# descriptors over convolutional layers.
# model = dc.models.GraphConvModel(n_tasks=1, mode="regression", dropout=0.2)
# Graph convolutional model for regression
warnings.filterwarnings('ignore')
model = dc.models.GraphConvModel(
    n_tasks=1,       # Number of regression targets (e.g., predicting a single property)
    mode="regression",  # Task type (regression/classification)
    dropout=0.2      # Regularization to prevent overfitting
)

### Train model

We now need to train the model on the data set.  We simply give it the data set and tell it how many epochs of training to perform (that is, how many complete passes through the data to make).

In [ ]:
warnings.filterwarnings("ignore")
# Returns the average loss over the most recent checkpoint interval
model.fit(train_dataset, nb_epoch=200)

### Evaluate model

If everything has gone well, we should now have a fully trained model!  But do we?  To find out, we must evaluate the model on the test set.  We do that by selecting an evaluation metric and calling `evaluate()` on the model.  For this example, let's use the Pearson correlation, also known as r<sup>2</sup>, as our metric.  We can evaluate it on both the training set and test set.

In [ ]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(train_dataset, [metric], transformers))
print("Test set score:", model.evaluate(test_dataset, [metric], transformers))

Notice that it has a higher score on the training set than the test set.  Models usually perform better on the particular data they were trained on than they do on similar but independent data.  This is called "overfitting", and it is the reason it is essential to evaluate your model on an independent test set.

Our model still has quite respectable performance on the test set.  For comparison, a model that produced totally random outputs would have a correlation of 0, while one that made perfect predictions would have a correlation of 1.  Our model does quite well, so now we can use it to make predictions about other molecules we care about.

### Make predictions

Let's just use the first ten molecules from the test set.  For each one we print out the chemical structure (represented as a SMILES string) and the predicted log(solubility). To put these predictions in 
context, we print out the log(solubility) values from the test set as well.

In [ ]:
solubilities = model.predict_on_batch(test_dataset.X[:10])
for molecule, solubility, test_solubility in zip(
    test_dataset.ids, solubilities, test_dataset.y
):
    print(solubility, test_solubility, molecule)

## Conclusion

In this tutorial, we learned how to:

- Work with molecular data using DeepChem
- Build deep learning models for property prediction
- Process chemical structures for machine learning
- Make predictions about molecular solubility

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.

